In [1]:
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns

from rich import print

In [2]:
plt.style.use("seaborn-v0_8")

In [28]:
data_schema = {
    "CustomerID": pl.Int32,
    "Age": pl.Int32,
    "Gender": pl.String,
    "Tenure": pl.Int32,
    "Usage Frequency": pl.Int32,
    "Support Calls": pl.Int32,
    "Payment Delay": pl.Int32,
    "Subscription Type": pl.String,
    "Contract Length": pl.String,
    "Total Spend": pl.Float32,
    "Last Interaction": pl.Int32,
    "Churn": pl.Int32,
}

In [30]:
train_df = pl.scan_csv("../data/train.csv", schema=data_schema)
test_df = pl.scan_csv("../data/test.csv", schema=data_schema)

In [35]:
full_df = pl.concat([train_df, test_df])  # NOTE: this is for analysis only

In [36]:
full_df.collect().head()

CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
i32,i32,str,i32,i32,i32,i32,str,str,f32,i32,i32
2,30,"""Female""",39,14,5,18,"""Standard""","""Annual""",932.0,17,1
3,65,"""Female""",49,1,10,8,"""Basic""","""Monthly""",557.0,6,1
4,55,"""Female""",14,4,6,18,"""Basic""","""Quarterly""",185.0,3,1
5,58,"""Male""",38,21,7,7,"""Standard""","""Monthly""",396.0,29,1
6,23,"""Male""",32,20,5,8,"""Basic""","""Monthly""",617.0,20,1
